1. use mySQL to query the zillow database.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split

from env import host, user, password
import acquire
import prepare

2. you will need to use the properties_2017 and predictions_2017 tables.
    - you will want to limit your data to properties that were sold in may, june, july or august (date is in predictions_2017 table).
    - There are a ton of rows...so think about how you can limit your data set early to keep your query going! i.e. filter the dates!

In [2]:
sql_query = '''select *
    from properties_2017
    join predictions_2017 using(parcelid)
    where propertylandusetypeid = 260
    or 261 or 263 or 273 or 274 or 276 or 279
    and unitcnt = 1
    and transactiondate 
    like '2017-05-%%' 
    or '2017-06-%%';
    '''
connection = f'mysql+pymysql://{user}:{password}@{host}/zillow'
df = pd.read_sql(sql_query, connection)

3. you will want to gather the following information (and figure out which columns are most likely to give you this information)...


- square feet of the house ("living square feet"), 


In [3]:
#finishedsqauarefeet12
#finishedsqauarefeet13
#finishedsqauarefeet15
#finishedsqauarefeet50
#finishedsqauarefeet6



- number of bedrooms, 


In [4]:
#bedroomcnt

- number of bathrooms, 


In [5]:
#bathroomcnt

- the assess value of the house by the tax appraisal district ('taxvaluedollarcnt'...this will be your target variable in the project), 


In [6]:
#taxvaluedollarcnt

- 2-3 other variables you think will be useful (think about what you know about what drives home prices, and also the amount of non-null values available in selecting your extra variables.


- propertylandusedesc
- airconditioningdesc
- regionidzip
- yearbuilt

## Lets clean it

In [7]:
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01
4,10887214,1970746,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,119407.0,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,0.006940,2017-01-01


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77614 entries, 0 to 77613
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77614 non-null  int64  
 1   id                            77614 non-null  int64  
 2   airconditioningtypeid         25007 non-null  float64
 3   architecturalstyletypeid      207 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77580 non-null  float64
 6   bedroomcnt                    77580 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49810 non-null  float64
 9   calculatedbathnbr             76964 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6037 non-null   float64
 12  calculatedfinishedsquarefeet  77379 non-null  float64
 13  f

- What I want to do:
    - Delete columns:
        - `parcelid`
        - `architecturalstyletypeid`
        - `basementsqft`
        - `buildingqualitytypeid`
        - `calculatedbathnbr`
        - `decktypeid`
        - `finishedsquarefeet12`
        - `finishedsquarefeet13`
        - `finishedsquarefeet15`
        - `finishedsquarefeet50`
        - `finishedsquarefeet6`
        - `fips`
        - `garagetotalsqft`
        - `latitude`
        - `longitude`
        - `poolsizesum`
        - `pooltypeid10`
        - `pooltypeid2`
        - `pooltypeid7`
        - `finishedfloor1squarefeet`
        - `propertycountylandusecode`
        - `propertyzoningdesc`
        - `rawcensustractandblock`
        - `storytypeid`
        - `threequarterbathnbr`
        - `unitcnt`
        - `yardbuildingsqft17`
        - `yardbuildingsqft26`
        - `fireplaceflag`
        - `structuretaxvaluedollarcnt`
        - `taxvaluedollarcnt`
        - `assessmentyear`
        - `landtaxvaluedollarcnt`
        - `taxdelinquencyflag`
        - `taxdelinquencyyear`
        - `logerror`
        - `transactiondate`
    - Rename columns:
        - `bathroomcnt` to "bathrooms"
        - `bedroomcnt` to "bedrooms"
        - `calculatedfinishedsquarefeet` to "house_square_feet"
        - `fireplacecnt` to "fireplaces"
        - `fullbathcnt` to "full_baths"
        - `poolcnt` to "pools"
        - `roomcnt` to "rooms"
        - `regionidzip` to "zip_code"
    - Leave columns alone for now:
        - `garagecarcnt`
        - `hashottuborspa`
        - `lotsizesquarefeet`
        - `regionidcity`
        - `regionidcounty`
        - `regionidneighborhood`
        - `yearbuilt`
        - `numberofstories`
        - `taxamount`
        - `id`
        - `airconditioningtypeid`

In [9]:
df = df.drop(["parcelid", 'architecturalstyletypeid',
              'basementsqft', 'buildingqualitytypeid', 
              'calculatedbathnbr', 'decktypeid', 'finishedsquarefeet12', 
             'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
             'finishedsquarefeet6', 'garagetotalsqft', 
              'poolsizesum', 'pooltypeid10', 'pooltypeid2', 
             'pooltypeid7', 'finishedfloor1squarefeet', 'propertycountylandusecode', 
             'rawcensustractandblock', 'storytypeid', 'threequarterbathnbr', 
              'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'fireplaceflag', 
              'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 
             'landtaxvaluedollarcnt', 'taxdelinquencyflag', 'taxdelinquencyyear',
             'logerror', 'transactiondate', 'censustractandblock', 'propertyzoningdesc', 
             'buildingclasstypeid'], axis=1)

In [10]:
df.head()

,id,airconditioningtypeid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,hashottuborspa,...,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,typeconstructiontypeid,yearbuilt,numberofstories,taxamount,id
0,1727539,NaN,3.5,4.0,3100.0,6059.0,NaN,3.0,2.0,NaN,...,53571.0,1286.0,NaN,96978.0,0.0,NaN,1998.0,NaN,11013.72,0
1,1387261,NaN,1.0,2.0,1465.0,6111.0,1.0,1.0,1.0,NaN,...,13091.0,2061.0,NaN,97099.0,5.0,NaN,1967.0,1.0,5672.48,1
2,11677,NaN,2.0,3.0,1243.0,6059.0,NaN,2.0,2.0,NaN,...,21412.0,1286.0,NaN,97078.0,6.0,NaN,1962.0,1.0,6488.30,2
3,2288172,NaN,3.0,4.0,2376.0,6037.0,NaN,3.0,NaN,NaN,...,396551.0,3101.0,NaN,96330.0,0.0,NaN,1970.0,NaN,1777.51,3
4,1970746,1.0,3.0,3.0,1312.0,6037.0,NaN,3.0,NaN,NaN,...,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1964.0,NaN,1533.89,4


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77614 entries, 0 to 77613
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77614 non-null  int64  
 1   airconditioningtypeid         25007 non-null  float64
 2   bathroomcnt                   77580 non-null  float64
 3   bedroomcnt                    77580 non-null  float64
 4   calculatedfinishedsquarefeet  77379 non-null  float64
 5   fips                          77580 non-null  float64
 6   fireplacecnt                  8289 non-null   float64
 7   fullbathcnt                   76964 non-null  float64
 8   garagecarcnt                  25520 non-null  float64
 9   hashottuborspa                1539 non-null   float64
 10  heatingorsystemtypeid         49572 non-null  float64
 11  latitude                      77580 non-null  float64
 12  longitude                     77580 non-null  float64
 13  l

In [12]:
df = df.rename(columns={'bathroomcnt': 'bathrooms', 
                       'bedroomcnt': 'bedrooms', 
                       'calculatedfinishedsquarefeet': 'house_squarefeet', 
                       'fireplacecnt': 'fireplaces', 
                       'fullbathcnt': 'full_baths', 
                       'poolcnt': 'pools', 
                       'roomcnt': 'rooms',
                       'airconditioningtypeid': 'ac_id',
                       'buildingclasstypeid': 'building_id',
                       'heatingorsystemtypeid': 'heating_id',
                       'typeconstructiontypeid': 'construction_id'})

In [13]:
df.head()

,id,ac_id,bathrooms,bedrooms,house_squarefeet,fips,fireplaces,full_baths,garagecarcnt,hashottuborspa,...,regionidcity,regionidcounty,regionidneighborhood,regionidzip,rooms,construction_id,yearbuilt,numberofstories,taxamount,id
0,1727539,NaN,3.5,4.0,3100.0,6059.0,NaN,3.0,2.0,NaN,...,53571.0,1286.0,NaN,96978.0,0.0,NaN,1998.0,NaN,11013.72,0
1,1387261,NaN,1.0,2.0,1465.0,6111.0,1.0,1.0,1.0,NaN,...,13091.0,2061.0,NaN,97099.0,5.0,NaN,1967.0,1.0,5672.48,1
2,11677,NaN,2.0,3.0,1243.0,6059.0,NaN,2.0,2.0,NaN,...,21412.0,1286.0,NaN,97078.0,6.0,NaN,1962.0,1.0,6488.30,2
3,2288172,NaN,3.0,4.0,2376.0,6037.0,NaN,3.0,NaN,NaN,...,396551.0,3101.0,NaN,96330.0,0.0,NaN,1970.0,NaN,1777.51,3
4,1970746,1.0,3.0,3.0,1312.0,6037.0,NaN,3.0,NaN,NaN,...,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1964.0,NaN,1533.89,4


In [14]:
df.head()

,id,ac_id,bathrooms,bedrooms,house_squarefeet,fips,fireplaces,full_baths,garagecarcnt,hashottuborspa,...,regionidcity,regionidcounty,regionidneighborhood,regionidzip,rooms,construction_id,yearbuilt,numberofstories,taxamount,id
0,1727539,NaN,3.5,4.0,3100.0,6059.0,NaN,3.0,2.0,NaN,...,53571.0,1286.0,NaN,96978.0,0.0,NaN,1998.0,NaN,11013.72,0
1,1387261,NaN,1.0,2.0,1465.0,6111.0,1.0,1.0,1.0,NaN,...,13091.0,2061.0,NaN,97099.0,5.0,NaN,1967.0,1.0,5672.48,1
2,11677,NaN,2.0,3.0,1243.0,6059.0,NaN,2.0,2.0,NaN,...,21412.0,1286.0,NaN,97078.0,6.0,NaN,1962.0,1.0,6488.30,2
3,2288172,NaN,3.0,4.0,2376.0,6037.0,NaN,3.0,NaN,NaN,...,396551.0,3101.0,NaN,96330.0,0.0,NaN,1970.0,NaN,1777.51,3
4,1970746,1.0,3.0,3.0,1312.0,6037.0,NaN,3.0,NaN,NaN,...,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1964.0,NaN,1533.89,4


In [15]:
df.info()
# need to drop ac_id because there is only 32.14% are non null
# need to garagecarcnt because there is only

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77614 entries, 0 to 77613
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     77614 non-null  int64  
 1   ac_id                  25007 non-null  float64
 2   bathrooms              77580 non-null  float64
 3   bedrooms               77580 non-null  float64
 4   house_squarefeet       77379 non-null  float64
 5   fips                   77580 non-null  float64
 6   fireplaces             8289 non-null   float64
 7   full_baths             76964 non-null  float64
 8   garagecarcnt           25520 non-null  float64
 9   hashottuborspa         1539 non-null   float64
 10  heating_id             49572 non-null  float64
 11  latitude               77580 non-null  float64
 12  longitude              77580 non-null  float64
 13  lotsizesquarefeet      69322 non-null  float64
 14  pools                  16174 non-null  float64
 15  pr